In [ ]:
from pprint import pprint
import draco as drc

import altair as alt
from draco.renderer import AltairRenderer
from vega_datasets import data

d = drc.Draco()
renderer = AltairRenderer()
df = data.cars()


def change_column_name(df):
  df.columns = df.columns.str.lower()
  return df


# base_facts = [
#   *base_facts,
#   "entity(view,root,v0).",
#   "entity(mark,v0,m).",
#   "entity(encoding,m,e0).",
#   "attribute((encoding,field),e0,year).",
#   "entity(encoding,m,e1).",
#   "attribute((encoding,field),e1,horsepower).",
# ]

# model = next(draco.complete_spec(base_facts, 10))
# spec = answer_set_to_dict(model.answer_set)
# pprint(model)
# display(renderer.render(spec, df))

# @app.get("/")
df = change_column_name(df)
# make cylinders a nominal
df["cylinders"] = df["cylinders"].astype(str)

In [ ]:
df

In [ ]:
base_scheme = drc.schema_from_dataframe(df)
base_facts = drc.dict_to_facts(base_scheme)


def recommend_charts(
  spec: list[str],
  draco: drc.Draco,
  num: int = 5,
  labeler=lambda i: f"CHART {i+1}",
) -> dict[str, tuple[list[str], dict]]:
  # Dictionary to store the generated recommendations, keyed by chart name
  chart_specs = {}
  for i, model in enumerate(draco.complete_spec(spec, num)):
    chart_name = labeler(i)
    spec = drc.answer_set_to_dict(model.answer_set)
    chart_specs[chart_name] = drc.dict_to_facts(spec), spec

    print(chart_name)
    print(f"COST: {model.cost}")
    chart = renderer.render(spec=spec, data=df)
    # Adjust column-faceted chart size
    if (
      isinstance(chart, alt.FacetChart)
      and chart.facet.column is not alt.Undefined
    ):
      chart = chart.configure_view(continuousWidth=130, continuousHeight=130)
    display(chart)

  return chart_specs


In [ ]:
r = recommend_charts(
  spec=[
    *base_facts,
    "entity(view,root,v0).",
    "entity(mark,v0,m).",
    # "attribute((mark,type),m,rect).",
    "entity(encoding,m,e0).",
    "attribute((encoding,field),e0,horsepower).",
    # "attribute((encoding,binning),e0, 10).",
    # "entity(encoding,m,e1).",
    "attribute((encoding,field),e1,miles_per_gallon).",
    "entity(encoding,m,e1).",
    # "attribute((encoding,field),e2,displacement).",
    # "entity(encoding,m,e2).",
    # "attribute((encoding,aggregate),e1,mean).",
    # "attribute((encoding,field),e1,origin).",
    # ":- {entity(encoding,_,_)} > 3.",
    # ":- {entity(encoding,_,_)} < 3.",
    ":- {entity(mark,_,_)} != 1.",
  ],
  draco=d,
  num=5,
)